In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 8.25 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-06 20:20:30.246112: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 20:20:30.368582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 20:20:30.368619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 20:20:30.407297: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 20:20:30.453561: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 20:20:30.454479: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

In [8]:
import pickle
from itertools import product

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=2,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults('mse_sup', header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)



# Treinando com 25 dados


In [9]:
tester = Tester(run_times=25, dataset_run_times=10)

In [10]:
tester.Loop(sort_by='mse_sup',
            boundarie=2.0,
            hidden_sizes = [[30, 21], [21, 12], [33, 21]],
            regularizers=[0.02, 0.2, 0.5],
            learning_rate=[0.01, 0.1])

Testando combinacao1: Hidden Size=[30, 21], regularizer=0.02, learning_rate=0.01
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  3.6772713029986335
Stopped at epoch:  114
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.1593267209991609
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.20115836300101364
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.1615897439987748
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.17345808000027318
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.16117755500090425
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.20445443599965074
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_1_18,0.947327,0.716080,-2.151392,0.963280,0.903978,0.221059,1.898574,0.304111,0.159204,0.231658
model_1_1_17,0.946711,0.715417,-2.118604,0.963126,0.904495,0.223644,1.903009,0.300947,0.159875,0.230411
model_1_1_16,0.946079,0.714740,-2.085780,0.962962,0.905005,0.226296,1.907535,0.297780,0.160584,0.229182
model_1_1_15,0.945431,0.714049,-2.052932,0.962790,0.905506,0.229015,1.912153,0.294610,0.161332,0.227971
model_1_1_14,0.944767,0.713345,-2.020066,0.962608,0.906000,0.231803,1.916865,0.291438,0.162120,0.226779
model_1_1_13,0.944085,0.712626,-1.987192,0.962416,0.906486,0.234662,1.921674,0.288266,0.162951,0.225609
model_1_1_12,0.943387,0.711891,-1.954322,0.962215,0.906962,0.237594,1.926583,0.285094,0.163825,0.224460
model_1_1_11,0.942670,0.711142,-1.921463,0.962003,0.907429,0.240600,1.931594,0.281923,0.164744,0.223334
model_1_1_10,0.941936,0.710377,-1.888627,0.961780,0.907885,0.243683,1.936711,0.278754,0.165709,0.222232
model_1_1_9,0.941183,0.709596,-1.855827,0.961546,0.908331,0.246843,1.941936,0.275589,0.166722,0.221156


DataFrame salvo em ../results/metrics_1_1
+++++++++++ [1_2] | 1 ++++++++++++++++++
Elapsed time:  0.17037794700081577
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_2] | 2 ++++++++++++++++++
Elapsed time:  0.1638462980008626
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 3 ++++++++++++++++++
Elapsed time:  0.16038915200078918
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_2] | 4 ++++++++++++++++++
Elapsed time:  0.16223742399961338
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 5 ++++++++++++++++++
Elapsed time:  0.158843318999061
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 6 ++++++++++++++++++
Elapsed time:  0.18323172900090867
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_2] | 7 ++++++++++++++++++
Elapsed time:  0.187544782

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_2_24,0.958441,0.726638,0.957692,0.780186,0.938573,0.174415,1.827972,0.321525,0.206363,0.263944
model_1_2_23,0.958164,0.726360,0.957906,0.782862,0.939055,0.175578,1.829834,0.319900,0.203851,0.261875
model_1_2_22,0.957875,0.726071,0.958110,0.785545,0.939529,0.176789,1.831761,0.318347,0.201332,0.259840
model_1_2_21,0.957575,0.725773,0.958304,0.788234,0.939994,0.178048,1.833756,0.316871,0.198807,0.257839
model_1_2_20,0.957263,0.725465,0.958488,0.790930,0.940451,0.179358,1.835819,0.315473,0.196276,0.255874
model_1_2_19,0.956939,0.725146,0.958661,0.793631,0.940900,0.180719,1.837953,0.314155,0.193741,0.253948
model_1_2_18,0.956602,0.724816,0.958824,0.796335,0.941339,0.182134,1.840159,0.312921,0.191202,0.252062
model_1_2_17,0.956252,0.724474,0.958975,0.799042,0.941768,0.183602,1.842441,0.311775,0.188660,0.250217
model_1_2_16,0.955889,0.724122,0.959114,0.801752,0.942187,0.185126,1.844799,0.310718,0.186116,0.248417
model_1_2_15,0.955512,0.723757,0.959241,0.804463,0.942595,0.186708,1.847236,0.309753,0.183571,0.246662


...

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_8_6_0,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_22,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_21,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_20,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_19,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_18,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_17,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_16,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_15,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131
model_8_6_14,0.99142,0.783247,0.993706,1.0,0.999687,0.036009,1.449432,0.002262,3.860245e-12,0.001131


DataFrame salvo em ../results/metrics_8_6
+++++++++++ [8_7] | 1 ++++++++++++++++++
Elapsed time:  0.03192894700077886
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [8_7] | 2 ++++++++++++++++++
Elapsed time:  0.027182144998732838
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [8_7] | 3 ++++++++++++++++++
Elapsed time:  0.027866763999554678
Stopped at epoch:  0
1/1 [==============================] - 0s 18ms/step
+++++++++++ [8_7] | 4 ++++++++++++++++++
Elapsed time:  0.029300040001544403
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_7] | 5 ++++++++++++++++++
Elapsed time:  0.028883111999675748
Stopped at epoch:  0
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_7] | 6 ++++++++++++++++++
Elapsed time:  0.03017451199957577
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [8_7] | 7 ++++++++++++++++++
Elapsed time:  0.02

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_8_7_0,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_22,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_21,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_20,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_19,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_18,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_17,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_16,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_15,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923
model_8_7_14,0.99142,0.783247,0.97577,0.997837,0.982173,0.036009,1.449432,0.143584,0.002262,0.072923


DataFrame salvo em ../results/metrics_8_7
+++++++++++ [8_8] | 1 ++++++++++++++++++
Elapsed time:  0.032089334999909624
Stopped at epoch:  0
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_8] | 2 ++++++++++++++++++
Elapsed time:  0.029176931999245426
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_8] | 3 ++++++++++++++++++
Elapsed time:  0.04891903799943975
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [8_8] | 4 ++++++++++++++++++
Elapsed time:  0.029974486998980865
Stopped at epoch:  0
1/1 [==============================] - 0s 13ms/step
+++++++++++ [8_8] | 5 ++++++++++++++++++
Elapsed time:  0.02777540500028408
Stopped at epoch:  0
1/1 [==============================] - 0s 14ms/step
+++++++++++ [8_8] | 6 ++++++++++++++++++
Elapsed time:  0.034518201000537374
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [8_8] | 7 ++++++++++++++++++
Elapsed time:  0.02

: 